In [1]:
hex_to_char = {0: 'f', 1: 'g', 2: 'h', 3: 'i', 4: 'j', 5: 'k', 6: 'l', 7: 'm', 8: 'n', 9: 'o', 10: 'p', 11: 'q', 12: 'r', 13: 's', 14: 't', 15: 'u'}

#Convert the number to hexadecimal
def convert_hex(byte):
    return hex_to_char[byte//16]+ hex_to_char[byte%16]

#Convert ciphertext to ASCII
def convert_ASCII(s):
    c = chr(16*(ord(s[0]) - ord('f')) + ord(s[1]) - ord('f'))
    return c

#Convert 8 byte(16 characters) ciphertext to ASCII
def cipher_ASCII(cipher):
    new_plain = ""
    for i in range(0, len(cipher), 2):
        new_plain += convert_ASCII(cipher[i:i+2])
    return new_plain

In [ ]:
from pyfinite import ffield

# Initialize a 128x128 grid with all elements set to -1
exp_store = [[-1]*128 for i in range(128)]

#Finite field
F = ffield.FField(7)

# Calculate the exponentiation of 'n' raised to 'pow_' using a recursive approach
def exponentiation(n, pow_):
    # Check if the result for the given (n, pow_) pair is already computed and stored
    if exp_store[n][pow_] != -1:
        return exp_store[n][pow_]

    result = 0
    if pow_ == 0:
        result = 1
    elif pow_ == 1:
        result = n
    elif pow_%2 == 0:
        # Compute result recursively for even pow_
        sqrt_n = exponentiation(n, pow_>>1)
        result = F.Multiply(sqrt_n, sqrt_n)
    else:
        # Compute result recursively for odd pow_
        sqrt_n = exponentiation(n, pow_>>1)
        result = F.Multiply(sqrt_n, sqrt_n)
        result = F.Multiply(n, result)

    # Store the computed result in the memoization table
    exp_store[n][pow_] = result
    return result

#Add two vectors element wise using XOR operation
def addVectors (v1, v2):
    result = [0] * len(v1)
    for i, (e1, e2) in enumerate(zip(v1, v2)):
        result[i] = e1 ^ e2
    return result

#Multiply each element of vector by a scalar element
def scalarMultiplication (v, ele):
    result = [0] * len(v)
    for i, e in enumerate(v):
        result[i] = F.Multiply(e, ele)  # Assuming F.Multiply performs scalar multiplication
    return result

#Perform linear transformation of a list of vectors using a matrix
def LinearTransformation (mat, elist):
    result = [0] * len(mat[0])  # Initialize result vector
    for row, elem in zip(mat, elist):
        scaled_row = scalarMultiplication(row, elem)  # Perform scalar multiplication
        result = addVectors(scaled_row, result)  # Add scaled row to result vector
    return result

#Mimics the EAEAE operation using linear transformations and exponentiations
def EAEAE (plaintext, lin_mat, exp_mat):
   # Convert plaintext characters to ASCII values
    plaintext = [ord(c) for c in plaintext]

    # Initialize Output matrix
    output = [[0 for _ in range(8)] for _ in range(5)]

    # First stage: Exponentiation on plaintext elements
    for ind, elem in enumerate(plaintext):
        output[0][ind] = exponentiation(elem, exp_mat[ind])  # Assuming exponentiation function is defined

    # Second stage: Linear transformation on Output[0]
    output[1] = LinearTransformation(lin_mat, output[0])

    # Third stage: Exponentiation on Output[1] elements
    for ind, elem in enumerate(output[1]):
        output[2][ind] = exponentiation(elem, exp_mat[ind])

    # Fourth stage: Linear transformation on Output[2]
    output[3] = LinearTransformation(lin_mat, output[2])

    # Fifth stage: Exponentiation on Output[3] elements
    for ind, elem in enumerate(output[3]):
        output[4][ind] = exponentiation(elem, exp_mat[ind])

    return output[4]  # Return the final result vector

In [ ]:
# List to store all possible exponents for each index (from 0 to 7)
possible_exponents = [[] for _ in range(8)]

## List to store all possible diagonal values in a matrix (8x8)
possible_diagonal_values = [[[] for i in range(8)] for j in range(8)]

input_file = open("plain_texts.txt", 'r')
output_file = open("cipher_texts.txt", 'r')

# Iterate through each line of input and output files
for index, (in_line, out_line) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    input_values = []
    output_values = []

    # Convert each input hex value to corresponding ASCII character
    for hex_input in in_line.strip().split(" "):
        input_values.append(cipher_ASCII(hex_input)[index])

    # Convert each output hex value to corresponding ASCII character
    for hex_output in out_line.strip().split(" "):
        output_values.append(cipher_ASCII(hex_output)[index])

    # Iterate over possible values of 'i' and 'j' (ranging from 1 to 126 and 1 to 127 respectively) 
    for i in range(1, 127):
        for j in range(1, 128):
            flag = True

            # Check if the given transformation holds for all input-output pairs
            for inputs, outputs in zip(input_values, output_values):
                
                # Calculate the transformed value based on the given exponents and multiplications
                transformed_value = exponentiation(F.Multiply(exponentiation(F.Multiply(exponentiation(ord(inputs), i), j), i), j), i)
                
                # If the transformed value does not match the expected output, set flag to False
                if ord(outputs) != transformed_value:
                    flag = False
                    break
            # If the transformation holds for all input-output pairs, store the values in possible lists
            if flag:
                
                possible_exponents[index].append(i)
                possible_diagonal_values[index][index].append(j)
print(possible_exponents)
print(possible_diagonal_values)

[[1, 19, 107], [39, 106, 109], [22, 37, 68], [4, 47, 76], [40, 89, 125], [9, 44, 74], [23, 48, 56], [18, 21, 88]]


[[[27, 84, 84], [], [], [], [], [], [], []],
 [[], [18, 70, 10], [], [], [], [], [], []],
 [[], [], [33, 43, 98], [], [], [], [], []],
 [[], [], [], [38, 113, 12], [], [], [], []],
 [[], [], [], [], [118, 112, 20], [], [], []],
 [[], [], [], [], [], [31, 11, 12], [], []],
 [[], [], [], [], [], [], [27, 71, 92], []],
 [[], [], [], [], [], [], [], [93, 38, 5]]]

In [4]:
input_file = open("plain_texts.txt", 'r')
output_file = open("cipher_texts.txt", 'r')

# Iterate through each line of plaintext and ciphertext files
for index, (in_line, out_line) in enumerate(zip(input_file.readlines(), output_file.readlines())):

    # Considering only the first 6 pairs (indices 0 to 5)
    if index > 6 :
        break
    
    # Initialize lists to store ASCII representations of input and output strings
    input_values = []
    output_values = []

    # Convert each input hex value to corresponding ASCII character
    for hex_input in in_line.strip().split(" "):
        input_values.append(cipher_ASCII(hex_input)[index]) 

    # Convert each output hex value to corresponding ASCII character
    for hex_output in out_line.strip().split(" "):
        output_values.append(cipher_ASCII(hex_output)[index+1])

    # Iterate over all possible pairs of exponents and diagonal values
    for i in range(1, 128):
        for p1, e1 in zip(possible_exponents[index+1], possible_diagonal_values[index+1][index+1]):
            for p2, e2 in zip(possible_exponents[index], possible_diagonal_values[index][index]):
                flag = True

                # Check the validity of the transformation for each input-output pair
                for inp, outp in zip(input_values, output_values):
                    # Calculate the transformed value using specified exponents and diagonal values
                    transformed_value = exponentiation(
                        (F.Multiply(exponentiation(F.Multiply(exponentiation(ord(inp), p2), e2), p2), i) 
                         ^ F.Multiply(exponentiation(F.Multiply(exponentiation(ord(inp), p2), i), p1), e1)), p1)
                    
                    # If the transformed value does not match the expected output, set flag to False
                    if ord(outp) != transformed_value:
                        flag = False
                        break
                
                # If the transformation holds for all input-output pairs, update the possible values
                if flag:
                    possible_exponents[index + 1] = [p1]
                    possible_diagonal_values[index + 1][index + 1] = [e1]
                    possible_exponents[index] = [p2]
                    possible_diagonal_values[index][index] = [e2]
                    possible_diagonal_values[index][index + 1] = [i]

input_file.close()
output_file.close()

print(possible_exponents)
print(possible_diagonal_values)

[[19], [106], [37], [76], [89], [44], [23], [21]]


[[[84], [115], [], [], [], [], [], []],
 [[], [70], [17], [], [], [], [], []],
 [[], [], [43], [], [], [], [], []],
 [[], [], [], [12], [110], [], [], []],
 [[], [], [], [], [112], [101], [], []],
 [[], [], [], [], [], [11], [93], []],
 [[], [], [], [], [], [], [27], [25]],
 [[], [], [], [], [], [], [], [38]]]

In [5]:
# Iterate over indices from 0 to 5 (total of 6 iterations)
for index in range(6):
    # Calculate the offset for diagonal elements skipping two positions every time
    offset = index + 2

    # Extract the list of exponents from possible_diagonal_values
    exp_list = [e[0] for e in possible_diagonal_values]

    # Initialize a linear transformation matrix (8x8) with default values of 0
    linear_transform_matrix = [[0 for _ in range(8)] for _ in range(8)]

    # Fill empty [] elements with 0 or first available value from possible_diagonal_values
    for i in range(8):
        for j in range(8):
            linear_transform_matrix[i][j] = 0 if len(possible_diagonal_values[i][j]) == 0 else possible_diagonal_values[i][j][0]

    input_file = open("plain_texts.txt", 'r')
    output_file = open("cipher_texts.txt", 'r')
    # Iterate over each line of input and output files
for index, (input_line, output_line) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    # Skip iterations where index exceeds (7 - offset)
    if index > (7 - offset):
        continue
    
    # Convert each input and output hex value to corresponding ASCII character
    input_chars = [cipher_ASCII(msg) for msg in input_line.strip().split(" ")]
    output_chars = [cipher_ASCII(msg) for msg in output_line.strip().split(" ")]
    
    # Iterate over all possible values of ai,j to find the one that satisfies the condition
    for value in range(1, 128):
        # Update the linear transformation matrix with the current value of ai,j
        linear_transform_matrix[index][index + offset] = value
        flag = True
        
        # Check if the resulting EAEAE output matches the expected output
        for input_char, output_char in zip(input_chars, output_chars):
            # Calculate EAEAE output and check against expected output character
            if EAEAE(input_char, linear_transform_matrix, exp_list)[index + offset] != ord(output_char[index + offset]):
                flag = False
                break
        
        # If the condition is satisfied for all input-output pairs, update possible_diagonal_values
        if flag:
            possible_diagonal_values[index][index + offset] = [value]

input_file.close()
output_file.close()

# Initialize a new linear transformation matrix with values from possible_diagonal_values
linear_transform_matrix = [[0 for _ in range(8)] for _ in range(8)]
for i in range(8):
    for j in range(8):
        linear_transform_matrix[i][j] = 0 if len(possible_diagonal_values[i][j]) == 0 else possible_diagonal_values[i][j][0]

print(exp_list)
print(linear_transform_matrix)

[19, 106, 37, 76, 89, 44, 23, 21]


[[84, 115, 26, 97, 98, 18, 15, 64],
 [0, 70, 17, 21, 33, 53, 120, 14],
 [0, 0, 43, 0, 3, 28, 0, 85],
 [0, 0, 0, 12, 110, 32, 104, 25],
 [0, 0, 0, 0, 112, 101, 30, 12],
 [0, 0, 0, 0, 0, 11, 93, 73],
 [0, 0, 0, 0, 0, 0, 27, 25],
 [0, 0, 0, 0, 0, 0, 0, 38]]

In [8]:
# Linear transformation matrix and exponent list obtained from the previous steps
LINEAR_TRANS = [[84, 115, 26, 97, 98, 18, 15, 64],
 [0, 70, 17, 21, 33, 53, 120, 14],
 [0, 0, 43, 0, 3, 28, 0, 85],
 [0, 0, 0, 12, 110, 32, 104, 25],
 [0, 0, 0, 0, 112, 101, 30, 12],
 [0, 0, 0, 0, 0, 11, 93, 73],
 [0, 0, 0, 0, 0, 0, 27, 25],
 [0, 0, 0, 0, 0, 0, 0, 38]]
EXPONENT = [19, 106, 37, 76, 89, 44, 23, 21]

In [9]:
# Two halves of the password
password1 = 'gsmgfnhmhtjtmqks'
password2 = 'moihgoiujifqhghk'

# Function to decrypt a password half by performing EAEAE with all possible 128 byte values
def DecryptPassword(password):
    password = cipher_ASCII(password) 
    output = ""  
    for i in range(8):
        for j in range(128):  # Iterate over all possible byte values (0 to 127)

            # Construct input by appending hex representation of 'ans' to the current output string
            input = output + convert_hex(j) + (16 - len(op) - 2) * 'f'

            # Check if the EAEAE output matches the corresponding password character
            if ord(password[i]) == EAEAE(cipher_ASCII(input), LINEAR_TRANS, EXPONENT)[i]:
                output += convert_hex(j) 
                break
    return output 

decrypted_password = cipher_ASCII(DecryptPassword(password1)) + cipher_ASCII(DecryptPassword(password2))
print(decrypted_password)

sknraqprdm000000
